# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [237]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

In [238]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [240]:
df = spaceship.dropna()
df1 = df.copy()
df1['Cabin'] = df['Cabin'].str[0]
df1.drop(columns=['PassengerId', 'Name'], axis=1, inplace=True)
bool_cols = ['CryoSleep', 'VIP', 'Transported']

for col in bool_cols:
    df1[col] = (df1[col] == True).astype(int)

df_cat = df1[['HomePlanet', 'Destination', 'Cabin']]

ohe = OneHotEncoder(sparse_output=False)

ohe.fit(df_cat)
cat_np = ohe.transform(df_cat)
cat_df = pd.DataFrame(cat_np, columns=ohe.get_feature_names_out(), index=df_cat.index)
df_num = df1.drop(columns=df_cat.columns)
df3 = pd.concat([df_num, cat_df], axis=1)


**Perform Train Test Split**

In [242]:
features = df3.drop(columns="Transported")
target = df3['Transported']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

x_train.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,...,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
3432,1,32.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7312,1,4.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2042,0,30.0,0,0.0,236.0,0.0,1149.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4999,0,17.0,0,13.0,0.0,565.0,367.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5755,1,26.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [243]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

print(f"Test Score: {knn.score(x_test, y_test)*100: .2f}%")
print(f"Train Score: {knn.score(x_train, y_train)*100: .2f}%")

Test Score:  77.23%
Train Score:  83.29%


In [244]:
#Now with min-max scaling:
from sklearn.preprocessing import MinMaxScaler

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

normalizer = MinMaxScaler()

normalizer.fit(x_train)

x_train_norm = normalizer.transform(x_train)
x_test_norm = normalizer.transform(x_test)

x_train_norm = pd.DataFrame(x_train_norm, columns=x_train.columns , index=x_train.index)
x_test_norm = pd.DataFrame(x_test_norm, columns=x_test.columns , index=x_test.index)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train_norm, y_train)

y_pred_knn = knn.predict(x_test_norm)

print(f"MAE: {mean_absolute_error(y_pred_knn, y_test): .3f}")
print(f"RMSE: {root_mean_squared_error(y_pred_knn, y_test): .3f}")
print(f"Test Score: {knn.score(x_test_norm, y_test)*100: .2f}%")
print(f"Train Score: {knn.score(x_train_norm, y_train)*100: .2f}%")

MAE:  0.239
RMSE:  0.489
Test Score:  76.10%
Train Score:  82.72%


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [247]:
#Normalised
bagging_clf = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)

bagging_clf.fit(x_train_norm, y_train)

y_pred_test_bag = bagging_clf.predict(x_test_norm)

print(f"MAE: {mean_absolute_error(y_pred_test_bag, y_test): .3f}")
print(f"RMSE: {root_mean_squared_error(y_pred_test_bag, y_test): .3f}")
print(f"Test Score: {bagging_clf.score(x_test_norm, y_test): .3f}")
print(f"Train Score: {bagging_clf.score(x_train_norm, y_train): .3f}")

MAE:  0.212
RMSE:  0.460
Test Score:  0.788
Train Score:  0.850


- Random Forests

In [249]:
#Normalised
forest = RandomForestClassifier(n_estimators=100,
                             max_depth=20)

forest.fit(x_train_norm, y_train)

y_pred_test_rf = forest.predict(x_test_norm)

print(f"MAE: {mean_absolute_error(y_pred_test_rf, y_test): .3f}")
print(f"RMSE: {root_mean_squared_error(y_pred_test_rf, y_test): .3f}")
print(f"Test Score: {forest.score(x_test_norm, y_test): .3f}")
print(f"Train Score: {forest.score(x_train_norm, y_train): .3f}")

MAE:  0.211
RMSE:  0.459
Test Score:  0.789
Train Score:  0.940


- Gradient Boosting

In [251]:
gb_clf = GradientBoostingClassifier(max_depth=20,
                                   n_estimators=100)

gb_clf.fit(x_train_norm, y_train)

y_pred_test_gb = gb_clf.predict(x_test_norm)

print(f"MAE: {mean_absolute_error(y_pred_test_gb, y_test): .3f}")
print(f"RMSE: {root_mean_squared_error(y_pred_test_gb, y_test): .3f}")
print(f"Test Score: {gb_clf.score(x_test_norm, y_test): .3f}")
print(f"Test Score: {gb_clf.score(x_train_norm, y_train): .3f}")

MAE:  0.213
RMSE:  0.462
Test Score:  0.787
Test Score:  0.940


- Adaptive Boosting

In [253]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
                            n_estimators=100)

ada_clf.fit(x_train_norm, y_train)

y_pred_test_ada = ada_clf.predict(x_test_norm)

print(f"MAE: {mean_absolute_error(y_pred_test_ada, y_test): .3f}")
print(f"RMSE: {root_mean_squared_error(y_pred_test_ada, y_test): .3f}")
print(f"Test Score: {ada_clf.score(x_test_norm, y_test): .3f}")
print(f"Train Score: {ada_clf.score(x_train_norm, y_train): .3f}")

C:\Users\olgeo\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


MAE:  0.232
RMSE:  0.482
Test Score:  0.768
Train Score:  0.940


Which model is the best and why?

In [255]:
#The ensemble methods were all strong than the KNN model score
# The results of each ensemble method were very similar, but the Random Forest was slightly the best
# It had the equal highest accuracy score for the test, and the best for the training data